In [1]:
import hashlib
import urllib2
import json
import os.path
from time import sleep
from datetime import timedelta, datetime, tzinfo

In [2]:
preprefix = "Datastream/"
prefix = "de.bayern.lfu/"
postfix = "/File/LiveMeasure"
urls = {u"Augsburg-BourgesPlatz-NO2":"https://www.lfu.bayern.de/_data_extern/luft/immissionsmessungen/csv/csv_706_172.csv",
        u"Augsburg_BourgesPlatz-PM10":"https://www.lfu.bayern.de/_data_extern/luft/immissionsmessungen/csv/csv_706_132G.csv",
       u"Augsburg-BourgesPlatz-O3":"https://www.lfu.bayern.de/_data_extern/luft/immissionsmessungen/csv/csv_706_180.csv",
       u"Augsburg-Karlstrasse-NO2":"https://www.lfu.bayern.de/_data_extern/luft/immissionsmessungen/csv/csv_1401_172.csv",
       u"Augsburg-Karlstrasse-PM10":"https://www.lfu.bayern.de/_data_extern/luft/immissionsmessungen/csv/csv_1401_132G.csv",
       u"Augsburg-Karlstrasse-O3":"https://www.lfu.bayern.de/_data_extern/luft/immissionsmessungen/csv/csv_1401_102.csv",
       u"Augsburg-Koenigsplatz-NO2":"https://www.lfu.bayern.de/_data_extern/luft/immissionsmessungen/csv/csv_701_172.csv",
       u"Augsburg-Koenigsplatz-PM10":"https://www.lfu.bayern.de/_data_extern/luft/immissionsmessungen/csv/csv_701_132G.csv",
       u"Augsburg-Koenigsplatz-O3":"https://www.lfu.bayern.de/_data_extern/luft/immissionsmessungen/csv/csv_701_102.csv",
       u"Augsburg-LfU-NO2":"https://www.lfu.bayern.de/_data_extern/luft/immissionsmessungen/csv/csv_1501_172.csv",
       u"Augsburg-LfU-PM10":"https://www.lfu.bayern.de/_data_extern/luft/immissionsmessungen/csv/csv_1501_132G.csv",
       u"Augsburg-LfU-O3":"https://www.lfu.bayern.de/_data_extern/luft/immissionsmessungen/csv/csv_1501_180.csv",
       u"Augsburg-LfU-CO":"https://www.lfu.bayern.de/_data_extern/luft/immissionsmessungen/csv/csv_1501_102.csv"}


aKey = urls.keys()[0]
#Get new file
print(urls[aKey])
response = urllib2.urlopen(urls[aKey])
newFile = response.read()
#Generate md5 for new file
mNew = hashlib.md5()
mNew.update(newFile)

#Get old file
oldFile = open("./Files/" + aKey, mode='r')

#Generate md5 for old file
mOld = hashlib.md5()
mOld.update(oldFile.read())
oldFile.close()

#Writefile if it changes
print(mNew.digest() == mOld.digest())
if(mNew.digest() != mOld.digest()):
    #Update the lastest
    writefile = open("./Files/" + aKey,'w')
    writefile.write(newFile)
    writefile.close()
    #Save with timestamp
    lmt = response.headers.getheader("Last-Modified")
    lmt = strftime('%Y-%m-%dT%H:%M:%SZ', strptime(lmt,"%a, %d %b %Y %H:%M:%S GMT"))
    writefile = open("./Files/{}.json".format(lmt),'w')
    writefile.write(newFile)
    writefile.close()
    #Save to Observation
    
    obsContent = {"@iot.id":lmt,"result":"urn"}
#    


In [ ]:

#aKey = urls.keys()[0]
while(True):
    for aKey in urls.keys():
        #Get new file
        directory = "./Files/{}".format(aKey)
        try:
            response = urllib2.urlopen(urls[aKey])
        except:
            #sleep(5)
            response.close()
            continue
        #Check if the file is already there
        lmt = response.headers.getheader("Last-Modified")
        lmt = datetime.strptime(lmt,
      "%a, %d %b %Y %H:%M:%S GMT")
        lmt = (lmt + timedelta(hours=2)).isoformat()
        
        obsContent = {}
        
        print(lmt)
        if not os.path.exists(directory):
            os.makedirs(directory)
        if(not os.path.isfile("{}/{}.json".format(directory, lmt))):
            #
            fetchedContent = response.read()
            response.close()
            # Create timestamp.json
            writefile = open("{}/{}.csv".format(directory, lmt),'w')
            writefile.write(fetchedContent)
            writefile.close()
            # Create timestamp.observation.json

            hasher = hashlib.sha1()
            hasher.update(fetchedContent)
            sha1 = hasher.hexdigest()

            obsContent = {"@iot.id":prefix + aKey + postfix + lmt,
                          "result":"urn:sha1:{}".format(sha1),
                          "phenomenonTime":lmt,
                          "resultTime":lmt,
                          "Datastream":{"@iot.id":preprefix + prefix + aKey + postfix},
                         
                          }
            '''
            "FeatureOfInterest": {
                            "name": aKey,
                            "description": "Location of {}".format(aKey),
                            "encodingType": "application/vnd.geo+json",
                            "feature": {
                              "type": "Point",
                              "coordinates": [0, 0]
                            }
            }
            '''
            writefile = open("{}/{}.csv.observation.json".format(directory,lmt),'w')
            writefile.write(json.dumps(obsContent))
            writefile.close()
            print("{} updated".format(aKey))
        else:
            print("No updates:{}".format(aKey))
    sleep(300)

2018-07-19T15:26:19
Augsburg-Karlstrasse-NO2 updated
2018-07-19T15:26:20
Augsburg-BourgesPlatz-O3 updated
2018-07-19T15:26:19
Augsburg-LfU-CO updated
2018-07-19T15:26:19
Augsburg-LfU-NO2 updated
2018-07-19T15:26:19
Augsburg-Karlstrasse-O3 updated
2018-07-19T15:26:19
Augsburg-LfU-O3 updated
2018-07-19T15:26:20
Augsburg_BourgesPlatz-PM10 updated
2018-07-19T15:26:20
Augsburg-Koenigsplatz-NO2 updated
2018-07-19T15:26:19
Augsburg-Karlstrasse-PM10 updated
2018-07-19T15:26:20
Augsburg-BourgesPlatz-NO2 updated
2018-07-19T15:26:20
Augsburg-Koenigsplatz-O3 updated
2018-07-19T15:26:20
Augsburg-Koenigsplatz-PM10 updated
2018-07-19T15:26:19
Augsburg-LfU-PM10 updated
2018-07-19T15:26:19
Augsburg-Karlstrasse-NO2 updated
2018-07-19T15:26:20
Augsburg-BourgesPlatz-O3 updated
2018-07-19T15:26:19
Augsburg-LfU-CO updated
2018-07-19T15:26:19
Augsburg-LfU-NO2 updated
2018-07-19T15:26:19
Augsburg-Karlstrasse-O3 updated
2018-07-19T15:26:19
Augsburg-LfU-O3 updated
2018-07-19T15:26:20
Augsburg_BourgesPlatz-PM10 

2018-07-19T16:26:15
Augsburg-Koenigsplatz-NO2 updated
2018-07-19T16:26:13
Augsburg-Karlstrasse-PM10 updated
2018-07-19T16:26:15
Augsburg-BourgesPlatz-NO2 updated
2018-07-19T16:26:15
Augsburg-Koenigsplatz-O3 updated
2018-07-19T16:26:15
Augsburg-Koenigsplatz-PM10 updated
2018-07-19T16:26:13
Augsburg-LfU-PM10 updated
2018-07-19T16:26:13
Augsburg-Karlstrasse-NO2 updated
2018-07-19T16:26:15
Augsburg-BourgesPlatz-O3 updated
2018-07-19T16:26:13
Augsburg-LfU-CO updated
2018-07-19T16:26:13
Augsburg-LfU-NO2 updated
2018-07-19T16:26:13
Augsburg-Karlstrasse-O3 updated
2018-07-19T16:26:13
Augsburg-LfU-O3 updated
2018-07-19T16:26:15
Augsburg_BourgesPlatz-PM10 updated
2018-07-19T16:26:15
Augsburg-Koenigsplatz-NO2 updated
2018-07-19T16:26:13
Augsburg-Karlstrasse-PM10 updated
2018-07-19T16:26:15
Augsburg-BourgesPlatz-NO2 updated
2018-07-19T16:26:15
Augsburg-Koenigsplatz-O3 updated
2018-07-19T16:26:15
Augsburg-Koenigsplatz-PM10 updated
2018-07-19T16:26:13
Augsburg-LfU-PM10 updated
2018-07-19T16:26:13
Au

2018-07-19T17:26:27
Augsburg-Karlstrasse-NO2 updated
2018-07-19T17:26:41
Augsburg-BourgesPlatz-O3 updated
2018-07-19T17:26:30
Augsburg-LfU-CO updated
2018-07-19T17:26:30
Augsburg-LfU-NO2 updated
2018-07-19T17:26:27
Augsburg-Karlstrasse-O3 updated
2018-07-19T17:26:30
Augsburg-LfU-O3 updated
2018-07-19T17:26:40
Augsburg_BourgesPlatz-PM10 updated
2018-07-19T17:26:39
Augsburg-Koenigsplatz-NO2 updated
2018-07-19T17:26:27
Augsburg-Karlstrasse-PM10 updated
2018-07-19T17:26:40
Augsburg-BourgesPlatz-NO2 updated
2018-07-19T17:26:38
Augsburg-Koenigsplatz-O3 updated
2018-07-19T17:26:39
Augsburg-Koenigsplatz-PM10 updated
2018-07-19T17:26:30
Augsburg-LfU-PM10 updated
2018-07-19T17:26:27
Augsburg-Karlstrasse-NO2 updated
2018-07-19T17:26:41
Augsburg-BourgesPlatz-O3 updated
2018-07-19T17:26:30
Augsburg-LfU-CO updated
2018-07-19T17:26:30
Augsburg-LfU-NO2 updated
2018-07-19T17:26:27
Augsburg-Karlstrasse-O3 updated
2018-07-19T17:26:30
Augsburg-LfU-O3 updated
2018-07-19T17:26:40
Augsburg_BourgesPlatz-PM10 

2018-07-20T09:26:14
Augsburg_BourgesPlatz-PM10 updated
2018-07-20T09:26:13
Augsburg-Koenigsplatz-NO2 updated
2018-07-20T09:26:12
Augsburg-Karlstrasse-PM10 updated
2018-07-20T09:26:14
Augsburg-BourgesPlatz-NO2 updated
2018-07-20T09:26:13
Augsburg-Koenigsplatz-O3 updated
2018-07-20T09:26:13
Augsburg-Koenigsplatz-PM10 updated
2018-07-20T09:26:13
Augsburg-LfU-PM10 updated
2018-07-20T09:26:12
Augsburg-Karlstrasse-NO2 updated
2018-07-20T09:26:14
Augsburg-BourgesPlatz-O3 updated
2018-07-20T09:26:13
Augsburg-LfU-CO updated
2018-07-20T09:26:13
Augsburg-LfU-NO2 updated
2018-07-20T09:26:12
Augsburg-Karlstrasse-O3 updated
2018-07-20T09:26:13
Augsburg-LfU-O3 updated
2018-07-20T09:26:14
Augsburg_BourgesPlatz-PM10 updated
2018-07-20T09:26:13
Augsburg-Koenigsplatz-NO2 updated
2018-07-20T09:26:12
Augsburg-Karlstrasse-PM10 updated
2018-07-20T09:26:14
Augsburg-BourgesPlatz-NO2 updated
2018-07-20T09:26:13
Augsburg-Koenigsplatz-O3 updated
2018-07-20T09:26:13
Augsburg-Koenigsplatz-PM10 updated
2018-07-20T09

2018-07-20T10:26:11
Augsburg-LfU-PM10 updated
2018-07-20T10:26:11
Augsburg-Karlstrasse-NO2 updated
2018-07-20T10:26:12
Augsburg-BourgesPlatz-O3 updated
2018-07-20T10:26:11
Augsburg-LfU-CO updated
2018-07-20T10:26:11
Augsburg-LfU-NO2 updated
2018-07-20T10:26:11
Augsburg-Karlstrasse-O3 updated
2018-07-20T10:26:11
Augsburg-LfU-O3 updated
2018-07-20T10:26:12
Augsburg_BourgesPlatz-PM10 updated
2018-07-20T10:26:12
Augsburg-Koenigsplatz-NO2 updated
2018-07-20T10:26:11
Augsburg-Karlstrasse-PM10 updated
2018-07-20T10:26:12
Augsburg-BourgesPlatz-NO2 updated
2018-07-20T10:26:12
Augsburg-Koenigsplatz-O3 updated
2018-07-20T10:26:12
Augsburg-Koenigsplatz-PM10 updated
2018-07-20T10:26:11
Augsburg-LfU-PM10 updated
2018-07-20T10:26:11
Augsburg-Karlstrasse-NO2 updated
2018-07-20T10:26:12
Augsburg-BourgesPlatz-O3 updated
2018-07-20T10:26:11
Augsburg-LfU-CO updated
2018-07-20T10:26:11
Augsburg-LfU-NO2 updated
2018-07-20T10:26:11
Augsburg-Karlstrasse-O3 updated
2018-07-20T10:26:11
Augsburg-LfU-O3 updated
2

2018-07-20T11:26:13
Augsburg_BourgesPlatz-PM10 updated
2018-07-20T11:26:13
Augsburg-Koenigsplatz-NO2 updated
2018-07-20T11:26:12
Augsburg-Karlstrasse-PM10 updated
2018-07-20T11:26:13
Augsburg-BourgesPlatz-NO2 updated
2018-07-20T11:26:13
Augsburg-Koenigsplatz-O3 updated
2018-07-20T11:26:13
Augsburg-Koenigsplatz-PM10 updated
2018-07-20T11:26:12
Augsburg-LfU-PM10 updated
2018-07-20T11:26:12
Augsburg-Karlstrasse-NO2 updated
2018-07-20T11:26:13
Augsburg-BourgesPlatz-O3 updated
2018-07-20T11:26:12
Augsburg-LfU-CO updated
2018-07-20T11:26:12
Augsburg-LfU-NO2 updated
2018-07-20T11:26:12
Augsburg-Karlstrasse-O3 updated
2018-07-20T11:26:12
Augsburg-LfU-O3 updated
2018-07-20T11:26:13
Augsburg_BourgesPlatz-PM10 updated
2018-07-20T11:26:13
Augsburg-Koenigsplatz-NO2 updated
2018-07-20T11:26:12
Augsburg-Karlstrasse-PM10 updated
2018-07-20T11:26:13
Augsburg-BourgesPlatz-NO2 updated
2018-07-20T11:26:13
Augsburg-Koenigsplatz-O3 updated
2018-07-20T11:26:13
Augsburg-Koenigsplatz-PM10 updated
2018-07-20T11

2018-07-20T12:26:13
Augsburg-LfU-PM10 updated
2018-07-20T12:26:13
Augsburg-Karlstrasse-NO2 updated
2018-07-20T12:26:19
Augsburg-BourgesPlatz-O3 updated
2018-07-20T12:26:13
Augsburg-LfU-CO updated
2018-07-20T12:26:13
Augsburg-LfU-NO2 updated
2018-07-20T12:26:13
Augsburg-Karlstrasse-O3 updated
2018-07-20T12:26:13
Augsburg-LfU-O3 updated
2018-07-20T12:26:19
Augsburg_BourgesPlatz-PM10 updated
2018-07-20T12:26:18
Augsburg-Koenigsplatz-NO2 updated
2018-07-20T12:26:13
Augsburg-Karlstrasse-PM10 updated
2018-07-20T12:26:19
Augsburg-BourgesPlatz-NO2 updated
2018-07-20T12:26:18
Augsburg-Koenigsplatz-O3 updated
2018-07-20T12:26:18
Augsburg-Koenigsplatz-PM10 updated
2018-07-20T12:26:13
Augsburg-LfU-PM10 updated
2018-07-20T12:26:13
Augsburg-Karlstrasse-NO2 updated
2018-07-20T12:26:19
Augsburg-BourgesPlatz-O3 updated
2018-07-20T12:26:13
Augsburg-LfU-CO updated
2018-07-20T12:26:13
Augsburg-LfU-NO2 updated
2018-07-20T12:26:13
Augsburg-Karlstrasse-O3 updated
2018-07-20T12:26:13
Augsburg-LfU-O3 updated
2

2018-07-20T13:26:13
Augsburg-Koenigsplatz-NO2 updated
2018-07-20T13:26:12
Augsburg-Karlstrasse-PM10 updated
2018-07-20T13:26:13
Augsburg-BourgesPlatz-NO2 updated
2018-07-20T13:26:13
Augsburg-Koenigsplatz-O3 updated
2018-07-20T13:26:13
Augsburg-Koenigsplatz-PM10 updated
2018-07-20T13:26:12
Augsburg-LfU-PM10 updated
2018-07-20T13:26:12
Augsburg-Karlstrasse-NO2 updated
2018-07-20T13:26:13
Augsburg-BourgesPlatz-O3 updated
2018-07-20T13:26:12
Augsburg-LfU-CO updated
2018-07-20T13:26:12
Augsburg-LfU-NO2 updated
2018-07-20T13:26:12
Augsburg-Karlstrasse-O3 updated
2018-07-20T13:26:12
Augsburg-LfU-O3 updated
2018-07-20T13:26:13
Augsburg_BourgesPlatz-PM10 updated
2018-07-20T13:26:13
Augsburg-Koenigsplatz-NO2 updated
2018-07-20T13:26:12
Augsburg-Karlstrasse-PM10 updated
2018-07-20T13:26:13
Augsburg-BourgesPlatz-NO2 updated
2018-07-20T13:26:13
Augsburg-Koenigsplatz-O3 updated
2018-07-20T13:26:13
Augsburg-Koenigsplatz-PM10 updated
2018-07-20T13:26:12
Augsburg-LfU-PM10 updated
2018-07-20T13:26:12
Au

2018-07-20T14:26:16
Augsburg-Karlstrasse-PM10 updated
2018-07-20T14:26:21
Augsburg-BourgesPlatz-NO2 updated
2018-07-20T14:26:20
Augsburg-Koenigsplatz-O3 updated
2018-07-20T14:26:20
Augsburg-Koenigsplatz-PM10 updated
2018-07-20T14:26:17
Augsburg-LfU-PM10 updated
2018-07-20T14:26:16
Augsburg-Karlstrasse-NO2 updated
2018-07-20T14:26:21
Augsburg-BourgesPlatz-O3 updated
2018-07-20T14:26:17
Augsburg-LfU-CO updated
2018-07-20T14:26:17
Augsburg-LfU-NO2 updated
2018-07-20T14:26:16
Augsburg-Karlstrasse-O3 updated
2018-07-20T14:26:17
Augsburg-LfU-O3 updated
2018-07-20T14:26:21
Augsburg_BourgesPlatz-PM10 updated
2018-07-20T14:26:20
Augsburg-Koenigsplatz-NO2 updated
2018-07-20T14:26:16
Augsburg-Karlstrasse-PM10 updated
2018-07-20T14:26:21
Augsburg-BourgesPlatz-NO2 updated
2018-07-20T14:26:20
Augsburg-Koenigsplatz-O3 updated
2018-07-20T14:26:20
Augsburg-Koenigsplatz-PM10 updated
2018-07-20T14:26:17
Augsburg-LfU-PM10 updated
2018-07-20T14:26:16
Augsburg-Karlstrasse-NO2 updated
2018-07-20T14:26:21
Aug

In [38]:
aKey = urls.keys()[0]
response = urllib2.urlopen(urls[aKey])
lmt = response.headers.getheader("Last-Modified")
dt = datetime.strptime(lmt,
      "%a, %d %b %Y %H:%M:%S GMT")
dt.isoformat

In [44]:
(dt + timedelta(hours=2)).isoformat()

'2018-07-17T12:26:22'

In [8]:

lmt = strftime('%Y-%m-%dT%H:%M:%SZ', strptime(lmt,
      "%a, %d %b %Y %H:%M:%S GMT"))
lmt

'2018-07-17T10:26:22Z'

In [32]:




dt.astimezone(GMT2())

ValueError: astimezone() cannot be applied to a naive datetime

In [35]:
dt.tzinfo = GMT2()

AttributeError: attribute 'tzinfo' of 'datetime.datetime' objects is not writable

In [62]:
hashlib.algorithms

('md5', 'sha1', 'sha224', 'sha256', 'sha384', 'sha512')

In [73]:
mm = hashlib.sha1()
mm.update("Hello, world!")
mm.hexdigest()
#.decode("hex") == mm.digest()
mm.hexdigest()

'943a702d06f34599aee1f8da8ef9f7296031d699'